In [70]:
import pandas as pd
from sqlalchemy import create_engine
import pymysql
pymysql.install_as_MySQLdb()

In [109]:
sat17 = "sat17.csv"
sat17_df = pd.read_csv(sat17)
sat17_df.head()

,cds,Ccode,CDcode,Scode,rtype,sname,dname,cname,enroll12,NumTstTakr,NumCurrElaBenchmark,NumPreElaBenchmark,TotNumElaBenchmark,PctElaBenchmark,NumCurrMathBenchmark,NumPreMathBenchmark,TotNumMathBenchmark,PctMathBenchmark,TotNumBothBenchmark,PctBothBenchmark
0,0,0,0,0,X,NaN,NaN,NaN,484169,221433,144893,15103,159996,72.25,98504,13887,112391,50.76,107173,48.40
1,1000000000000,1,100000,0,C,NaN,NaN,Alameda,16638,9029,5407,1376,6783,75.12,4033,1337,5370,59.48,5147,57.01
2,1100170000000,1,110017,0,D,NaN,Alameda County Office of Education,Alameda,253,83,28,0,28,33.73,9,0,9,10.84,9,10.84
3,1100170112607,1,110017,112607,S,Envision Academy for Arts & Technology,Alameda County Office of Education,Alameda,81,83,28,0,28,33.73,9,0,9,10.84,9,10.84
4,1611190000000,1,161119,0,D,NaN,Alameda Unified,Alameda,910,495,373,51,424,85.66,298,47,345,69.70,330,66.67


In [110]:
county_file = "Income by county.csv"
county_file_df = pd.read_csv(county_file)
county_file_df.head()
county_file_df = county_file_df.replace({'Geography': r' County, California$'}, {'Geography': ''}, regex=True)

In [111]:
 # Create a filtered dataframe from specific columns
sat17_cols = ["cname", "TotNumBothBenchmark", "PctBothBenchmark"]
sat17_transformed= sat17_df[sat17_cols].copy()

In [112]:
# Rename the column headers
sat17_transformed = sat17_transformed.rename(columns={"cname": "County",
                                                          "TotNumBothBenchmark":"total_number_benchmark" ,
                                                          "PctBothBenchmark": "percentage_benchmark"})

In [113]:
# Clean the data by dropping duplicates and setting the index
#sat17_transformed.drop_duplicates("id", inplace=True)
#sat17_transformed.set_index("id", inplace=True)
#This was not needed
sat17_transformed.head()

,County,total_number_benchmark,percentage_benchmark
0,NaN,107173,48.40
1,Alameda,5147,57.01
2,Alameda,9,10.84
3,Alameda,9,10.84
4,Alameda,330,66.67


In [114]:
# Create a filtered dataframe from specific columns
county_cols = ["Geography", "Households; Estimate; Total", "Nonfamily households; Estimate; Mean income (dollars)"]
county_transformed = county_file_df[county_cols].copy()

In [115]:
# Rename the column headers
county_transformed = county_transformed.rename(columns={"Geography": "County",
                                                         "Households; Estimate; Total": "Households; Estimate; Total",
                                                         "Nonfamily households; Estimate; Mean income (dollars)": "mean_income"})

# Set index
#county_transformed.set_index("id", inplace=True)

county_transformed.head()

,County,Households; Estimate; Total,mean_income
0,Alameda,569070,73298
1,Alpine,304,47028
2,Amador,14185,39939
3,Butte,86167,41019
4,Calaveras,17812,43678


In [116]:
#Creation of a new database
connection_string = "root:tummyfloor@localhost/"
engine = create_engine(f'mysql://{connection_string}')

connection = engine.connect()

In [117]:
connection.execute('create schema sat_income_db')

ProgrammingError: (pymysql.err.ProgrammingError) (1007, "Can't create database 'sat_income_db'; database exists") [SQL: 'create schema sat_income_db'] (Background on this error at: http://sqlalche.me/e/f405)

In [121]:
db_connection = "root:tummyfloor@localhost/sat_income_db"
db_engine = create_engine(f'mysql://{db_connection}')

#confirm tables
db_engine.table_names()

[]

In [123]:
sat17_transformed.to_sql(name= 'SAT17', con = db_engine, if_exists= 'append', index=True)

In [124]:
county_transformed.to_sql(name='CountyIncome', con=db_engine, if_exists='append', index=True) 

In [125]:
db_engine.table_names()

['countyincome', 'sat17']

In [129]:
result = db_engine.execute('select countyincome.county, countyincome.mean_income as "Mean Income", sat17.total_number_benchmark, sat17.percentage_benchmark \
from countyincome inner join sat17 ON countyincome.county = sat17.county group by sat17.County;').fetchall()

In [141]:
result

[('Alameda', 73298, '5147', '57.01'),
 ('Alpine', 47028, None, None),
 ('Amador', 39939, '75', '72.12'),
 ('Butte', 41019, '436', '63.56'),
 ('Calaveras', 43678, '92', '69.70'),
 ('Colusa', 44955, '47', '40.87'),
 ('Contra Costa', 72354, '3543', '55.08'),
 ('Del Norte', 37519, '40', '65.57'),
 ('El Dorado', 56048, '803', '79.50'),
 ('Fresno', 45987, '2308', '31.98'),
 ('Glenn', 32669, '45', '34.62'),
 ('Humboldt', 39122, '256', '60.95'),
 ('Imperial', 36964, '380', '42.13'),
 ('Inyo', 44227, '41', '36.94'),
 ('Kern', 47741, '1839', '39.62'),
 ('Kings', 55963, '209', '33.71'),
 ('Lake', 36757, '83', '40.69'),
 ('Lassen', 46349, '48', '50.53'),
 ('Los Angeles', 65092, '24427', '39.20'),
 ('Madera', 45518, '215', '29.09'),
 ('Marin', 97009, '739', '69.32'),
 ('Mariposa', 41680, '26', '68.42'),
 ('Mendocino', 49952, '149', '51.74'),
 ('Merced', 43786, '515', '33.35'),
 ('Modoc', 31760, '22', '59.46'),
 ('Mono', 57955, '28', '56.00'),
 ('Monterey', 62313, '748', '38.22'),
 ('Napa', 73157, '

In [143]:
db_engine.execute("CREATE TABLE sat_to_income (county VARCHAR(255), mean_income VARCHAR(255), total_number_benchmark VARCHAR(255), percentage_benchmark VARCHAR(255))")

In [145]:
sql = "INSERT INTO sat_to_income (county, mean_income, total_number_benchmark, percentage_benchmark) VALUES (%s, %s,%s, %s)"
db_engine.execute(sql, result)